In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## L

In [3]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')
### Pretrained Word Embeddings

folder_path = "/content/drive/MyDrive/model_imp_6kdata/"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CPU times: user 4.25 s, sys: 743 ms, total: 4.99 s
Wall time: 9.03 s


## BECODEMIXED 2x Dataset kiiiiiiiiii

In [3]:
# %%time
# becodemixed_data =  pd.read_excel('/content/drive/MyDrive/model_imp_whole_2/be_data_no_un_combined.xlsx') 


In [4]:
%%time
train_data =  pd.read_excel('/content/drive/MyDrive/model_imp_6kdata/train.xlsx') 
valid_data = pd.read_excel('/content/drive/MyDrive/model_imp_6kdata/val.xlsx')
test_data = pd.read_excel('/content/drive/MyDrive/model_imp_6kdata/test.xlsx')

CPU times: user 619 ms, sys: 16 ms, total: 635 ms
Wall time: 2.76 s


In [5]:
# import random  

# def split_three_ways(dataset, ratio1, ratio2, ratio3):
#   # shuffle dataset
  
  
#   cp_dataset = dataset.sample(frac=1)
#   total = len(dataset)

#   d = ratio1 + ratio2 + ratio3
#   size_train = round(round(ratio1 / d, 2) * total)

#   size_rest = total - size_train

#   size_valid = round(round(ratio2 / (ratio2 + ratio3), 2) * size_rest)
#   size_test = size_rest - size_valid

#   train_data = cp_dataset[0:size_train]
#   valid_data = cp_dataset[size_train : size_train + size_valid]
#   test_data = cp_dataset[size_train + size_valid:]
#   return train_data, valid_data, test_data 

# train_data, valid_data, test_data = split_three_ways(becodemixed_data, 80, 10, 10)
# print(len(train_data), len(valid_data), len(test_data))


In [6]:
valid_data.head(2)

,Label,Text
0,Joy,A movie made on climbers of Everest summit was...
1,Anger,the Movie portrayed a tribal sect of a particu...


In [7]:
train_data['Label'].value_counts()

Joy         1361
Anger        897
Sadness      832
Surprise     820
Disgust      793
Fear         750
Name: Label, dtype: int64

In [8]:
train_data['enc_label'] = train_data['Label'].replace({'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})
valid_data['enc_label'] = valid_data['Label'].replace({'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})

In [9]:
train_data.head(1)

,Label,Text,enc_label
0,Joy,Jurassic Park er ekta sequel Jurassic World je...,0


In [10]:
test_data.head(1)

,Unnamed: 0,Text
0,0,tumi sagor er eto pare gele keno


In [11]:
# train_data['Text'].tolist()

In [12]:
train_data['enc_label'].value_counts()

0    1361
4     897
1     832
2     820
3     793
5     750
Name: enc_label, dtype: int64

In [13]:
valid_data['enc_label'].value_counts()

0    179
4    128
2    106
5     99
1     92
3     77
Name: enc_label, dtype: int64

In [14]:
valid_data['Label'].value_counts()

Joy         179
Anger       128
Surprise    106
Fear         99
Sadness      92
Disgust      77
Name: Label, dtype: int64

In [15]:
print("Size of the training data ==> ", train_data.shape)
print("Size of the validation data ==>", valid_data.shape)
print("Size of the Test data ==>", test_data.shape)

Size of the training data ==>  (5453, 3)
Size of the validation data ==> (681, 3)
Size of the Test data ==> (680, 2)


## One Hot Encoding

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [17]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(train_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

train_data= pd.concat([train_data, encoded_labels ], axis=1)

In [18]:
train_data = train_data.rename(columns={'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
train_data.head(1)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,Joy,Jurassic Park er ekta sequel Jurassic World je...,0,0.0,0.0,0.0,1.0,0.0,0.0


## Cleaning

In [19]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [20]:
def get_becodemixed_stop_words():
    #swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("/content/drive/MyDrive/model_imp_6kdata/custom_becodemixed_stopwords.txt",  header=None) 
    #sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    #tamil_stop_words = list(set(sw1 + sw2))
    becodemixed_stop_words = list(sw2)
    return becodemixed_stop_words

In [21]:
#removing URLs and Punctuations
#train_data['cleaned'] = train_data['tweet'].apply(remove_url)
#Removing punctuations


becodemixed_stop_words = get_becodemixed_stop_words()
valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))

train_data['cleaned'] = train_data['Texts'].apply(text_cleaning)
valid_data['cleaned'] = valid_data['Texts'].apply(text_cleaning)
test_data['cleaned'] = test_data['Texts'].apply(text_cleaning)


## Data samples after cleaning
print("Original Data:===\n",train_data.Text[0],"\nCleaned Data:===\n",train_data.cleaned[0],)

Original Data:===
 Jurassic Park er ekta sequel Jurassic World jekhane gengineered dinosaurs der dekhano hoyeche , etao ager film tar moton e awfully gripping r commercially successful . ---- 
Cleaned Data:===
 jurassic park sequel jurassic world gengineered dinosaurs der dekhano hoyeche etao ager film tar moton awfully gripping commercially successful


In [22]:
valid_data['cleaned'].shape

(681,)

##ML

In [23]:
def print_metrices(pred,true):
    print(confusion_matrix(true,pred))
    print(classification_report(true,pred,))
    print("Accuracy : ",accuracy_score(pred,true))
    print("Precison : ",precision_score(pred,true, average = 'weighted'))
    print("Recall : ",recall_score(pred,true,  average = 'weighted'))
    print("F1 : ",f1_score(pred,true,  average = 'weighted'))

In [24]:
## TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

tfidfvectorizer = TfidfVectorizer(max_features=40000,ngram_range=(1,2),tokenizer=lambda x: x.split())
tfidfvectorised = tfidfvectorizer.fit_transform(train_data['cleaned'])
tfidfvectorized_train = tfidfvectorizer.transform(train_data['cleaned'])
tfidfvectorized_valid = tfidfvectorizer.transform(valid_data['cleaned'])
tfidfvectorized_test = tfidfvectorizer.transform(test_data['cleaned'])

print("Feature Size: ", tfidfvectorized_train.shape[1])


Feature Size:  40000


In [25]:
tfidfvectorized_valid.shape

(681, 40000)

In [26]:
valid_data['Label'].value_counts()

Joy         179
Anger       128
Surprise    106
Fear         99
Sadness      92
Disgust      77
Name: Label, dtype: int64

In [27]:

actual_labels = pd.read_excel('/content/drive/MyDrive/model_imp_6kdata/test_with_label_forwhole.xlsx')

actual_labels 

,label,text
0,Anger,tumi sagor er eto pare gele keno
1,Joy,Your most favourite juti ? ? ? ? Pls vote guys...
2,Anger,etai mone o mitthe bolche 😒 😒 Dev Dev 😏
3,Disgust,"Ei movie ta dekhe mone holo , ami worst direct..."
4,Disgust,"eita bangladesh eigula hbei,ei desh e teke eig..."
...,...,...
675,Disgust,Nickleodeon animated series ta thik jototai na...
676,Anger,tomar kukur ta amader onek distrub kore
677,Sadness,Koto expekta equipmpuro kelasness er jonho kha...
678,Sadness,Ami gorbito ami bangladeshi ai desh e somvob a...


In [28]:
labels = ['Joy','Sadness','Surprise','Disgust','Anger','Fear']
def actual_label(row):
    return labels[row]

### Logistic Regression

In [29]:
# lr = LogisticRegression(class_weight='balanced',C =10)
lr = LogisticRegressionCV(class_weight='balanced',max_iter=500)
lr.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = lr.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[121  18  13  22   4   1]
 [ 25  43   2  14   8   0]
 [ 22   4  55   7  12   6]
 [ 11  11   1  45   9   0]
 [  7  14  16  10  75   6]
 [  4  11   8   6  10  60]]
              precision    recall  f1-score   support

           0       0.64      0.68      0.66       179
           1       0.43      0.47      0.45        92
           2       0.58      0.52      0.55       106
           3       0.43      0.58      0.50        77
           4       0.64      0.59      0.61       128
           5       0.82      0.61      0.70        99

    accuracy                           0.59       681
   macro avg       0.59      0.57      0.58       681
weighted avg       0.60      0.59      0.59       681

Accuracy :  0.5859030837004405
Precison :  0.586045519502068
Recall :  0.5859030837004405
F1 :  0.5817862502518499


In [30]:
test_pred = lr.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [31]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Anger
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Disgust
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Disgust
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Disgust
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Disgust
676,677,tomar kukur ta amader onek distrub kore,Anger
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Sadness
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Sadness


In [32]:
df['label'].value_counts()

Joy         161
Sadness     115
Anger       112
Disgust     111
Surprise     91
Fear         90
Name: label, dtype: int64

In [33]:
df.to_csv(folder_path +'becodemixed_logisticRegression.tsv',sep = '\t',index = False)

In [34]:
print_metrices(actual_labels['label'],df['label'])

[[ 60   8  12   9   7  16]
 [  7  50   1  19  18  16]
 [  6   1  69   3   1  10]
 [  6  11   3 112  18  11]
 [ 10  15   4  19  58   9]
 [ 12   3   7   5   4  60]]
              precision    recall  f1-score   support

       Anger       0.59      0.54      0.56       112
     Disgust       0.57      0.45      0.50       111
        Fear       0.72      0.77      0.74        90
         Joy       0.67      0.70      0.68       161
     Sadness       0.55      0.50      0.52       115
    Surprise       0.49      0.66      0.56        91

    accuracy                           0.60       680
   macro avg       0.60      0.60      0.60       680
weighted avg       0.60      0.60      0.60       680

Accuracy :  0.6014705882352941
Precison :  0.6138545963897626
Recall :  0.6014705882352941
F1 :  0.6040698593143106


## DT

In [35]:
dt = DecisionTreeClassifier(class_weight='balanced')
dt.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = dt.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[84 23 33 27  8  4]
 [15 38  7 11 18  3]
 [14  5 59  7 16  5]
 [24 11  9 17 12  4]
 [ 8  8 34  9 61  8]
 [ 4  7 16  7 15 50]]
              precision    recall  f1-score   support

           0       0.56      0.47      0.51       179
           1       0.41      0.41      0.41        92
           2       0.37      0.56      0.45       106
           3       0.22      0.22      0.22        77
           4       0.47      0.48      0.47       128
           5       0.68      0.51      0.58        99

    accuracy                           0.45       681
   macro avg       0.45      0.44      0.44       681
weighted avg       0.47      0.45      0.46       681

Accuracy :  0.45374449339207046
Precison :  0.4587559846261804
Recall :  0.45374449339207046
F1 :  0.44977297709594916


In [36]:
test_pred = dt.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [37]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Anger
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Sadness
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Fear
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Surprise
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Disgust
676,677,tomar kukur ta amader onek distrub kore,Anger
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Disgust
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Joy


In [38]:
print_metrices(actual_labels['label'],df['label'])

[[42 15 19 21 19 19]
 [ 9 24  4 33 14  8]
 [ 8  7 53 10  3  8]
 [ 1 19  4 58 19 16]
 [ 9 11  1 19 30  9]
 [32 12 15 26 21 62]]
              precision    recall  f1-score   support

       Anger       0.42      0.31      0.36       135
     Disgust       0.27      0.26      0.27        92
        Fear       0.55      0.60      0.57        89
         Joy       0.35      0.50      0.41       117
     Sadness       0.28      0.38      0.32        79
    Surprise       0.51      0.37      0.43       168

    accuracy                           0.40       680
   macro avg       0.40      0.40      0.39       680
weighted avg       0.41      0.40      0.40       680

Accuracy :  0.39558823529411763
Precison :  0.4111920325798407
Recall :  0.39558823529411763
F1 :  0.39584762917623


### SVM

In [39]:
# C : float, default=1.0
#     Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive
# The "balanced" mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).

In [40]:
## SVM
from sklearn.svm import SVC
svc =  SVC(C= 6, kernel='rbf' ,random_state=265,class_weight='balanced')
svc.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = svc.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[145   8   6  18   2   0]
 [ 32  39   2  12   7   0]
 [ 32   4  49   7   9   5]
 [ 20   4   1  46   6   0]
 [ 20  14  13   6  71   4]
 [  9   9   9   4  10  58]]
              precision    recall  f1-score   support

           0       0.56      0.81      0.66       179
           1       0.50      0.42      0.46        92
           2       0.61      0.46      0.53       106
           3       0.49      0.60      0.54        77
           4       0.68      0.55      0.61       128
           5       0.87      0.59      0.70        99

    accuracy                           0.60       681
   macro avg       0.62      0.57      0.58       681
weighted avg       0.62      0.60      0.60       681

Accuracy :  0.5991189427312775
Precison :  0.6344991441785082
Recall :  0.5991189427312775
F1 :  0.6024841303318469


In [41]:
test_pred = svc.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [42]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Joy
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Sadness
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Disgust
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Sadness
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Disgust
676,677,tomar kukur ta amader onek distrub kore,Anger
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Sadness
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Sadness


In [43]:
df['label'].value_counts()

Joy         235
Anger       122
Sadness      90
Disgust      87
Surprise     74
Fear         72
Name: label, dtype: int64

In [44]:
df.to_csv(folder_path +'becodemixed_svm.tsv',sep = '\t',index = False)

In [45]:
print_metrices(actual_labels['label'],df['label'])

[[ 64   7  16   6  10  19]
 [  3  45   2  11  15  11]
 [  4   0  59   1   1   7]
 [ 14  21   6 133  35  26]
 [  7  14   4  13  45   7]
 [  9   1   9   3   0  52]]
              precision    recall  f1-score   support

       Anger       0.63      0.52      0.57       122
     Disgust       0.51      0.52      0.51        87
        Fear       0.61      0.82      0.70        72
         Joy       0.80      0.57      0.66       235
     Sadness       0.42      0.50      0.46        90
    Surprise       0.43      0.70      0.53        74

    accuracy                           0.59       680
   macro avg       0.57      0.60      0.57       680
weighted avg       0.62      0.59      0.59       680

Accuracy :  0.5852941176470589
Precison :  0.603547261691331
Recall :  0.5852941176470589
F1 :  0.5802492048288428


## MNB

In [46]:
from sklearn.naive_bayes import MultinomialNB
nb =   MultinomialNB(alpha=0.1)#alpha : float, default=1.0 additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).
nb.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = nb.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[144   9   7  16   2   1]
 [ 28  36   1  17   9   1]
 [ 28   3  45   7  16   7]
 [ 15   9   0  44   9   0]
 [ 19  14  10   8  69   8]
 [  9   6   9   5  12  58]]
              precision    recall  f1-score   support

           0       0.59      0.80      0.68       179
           1       0.47      0.39      0.43        92
           2       0.62      0.42      0.51       106
           3       0.45      0.57      0.51        77
           4       0.59      0.54      0.56       128
           5       0.77      0.59      0.67        99

    accuracy                           0.58       681
   macro avg       0.58      0.55      0.56       681
weighted avg       0.59      0.58      0.58       681

Accuracy :  0.5814977973568282
Precison :  0.6147148074977624
Recall :  0.5814977973568282
F1 :  0.587382676578222


In [47]:
test_pred = nb.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index

In [48]:
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,0,Anger
1,1,Anger
2,2,Sadness
3,3,Disgust
4,4,Anger
...,...,...
675,675,Disgust
676,676,Anger
677,677,Sadness
678,678,Fear


In [49]:
df['label'].value_counts()

Joy         215
Anger       123
Sadness      98
Fear         91
Disgust      88
Surprise     65
Name: label, dtype: int64

In [50]:
df.to_csv(folder_path +'becodemixed_mnbfinal.tsv',sep = '\t',index = False)

In [51]:
print_metrices(actual_labels['label'],df['label'])

[[ 62   8  11   7   8  27]
 [  4  45   2   8  18  11]
 [  6   2  71   2   1   9]
 [  8  22   2 135  27  21]
 [ 12   9   4  13  51   9]
 [  9   2   6   2   1  45]]
              precision    recall  f1-score   support

       Anger       0.61      0.50      0.55       123
     Disgust       0.51      0.51      0.51        88
        Fear       0.74      0.78      0.76        91
         Joy       0.81      0.63      0.71       215
     Sadness       0.48      0.52      0.50        98
    Surprise       0.37      0.69      0.48        65

    accuracy                           0.60       680
   macro avg       0.59      0.61      0.59       680
weighted avg       0.64      0.60      0.61       680

Accuracy :  0.6014705882352941
Precison :  0.610730789545191
Recall :  0.6014705882352941
F1 :  0.5934740246505971


#Random Forest

In [52]:
rf = RandomForestClassifier(n_estimators= 100, class_weight='balanced')
rf.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = rf.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[107  11  45  12   4   0]
 [ 23  29  15  11  12   2]
 [ 17   0  74   7   5   3]
 [ 25   5   9  29   9   0]
 [  7  10  37   4  66   4]
 [  6   0  23   1  13  56]]
              precision    recall  f1-score   support

           0       0.58      0.60      0.59       179
           1       0.53      0.32      0.39        92
           2       0.36      0.70      0.48       106
           3       0.45      0.38      0.41        77
           4       0.61      0.52      0.56       128
           5       0.86      0.57      0.68        99

    accuracy                           0.53       681
   macro avg       0.57      0.51      0.52       681
weighted avg       0.57      0.53      0.53       681

Accuracy :  0.5301027900146843
Precison :  0.5678637704906879
Recall :  0.5301027900146843
F1 :  0.5273414622544974


In [53]:
test_pred = rf.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [54]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Surprise
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Sadness
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Fear
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Joy
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Sadness
676,677,tomar kukur ta amader onek distrub kore,Surprise
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Anger
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Sadness


In [55]:
df['label'].value_counts()

Surprise    202
Joy         164
Anger       105
Fear         73
Disgust      72
Sadness      64
Name: label, dtype: int64

In [56]:
df.to_csv(folder_path +'becodemixed_rf.tsv',sep = '\t',index = False)

In [57]:
print_metrices(actual_labels['label'],df['label'])

[[ 48  10  13  13  14   7]
 [  3  33   1  13  15   7]
 [  5   4  55   1   3   5]
 [  3  21   3 100  20  17]
 [  6   9   1  10  34   4]
 [ 36  11  23  30  20  82]]
              precision    recall  f1-score   support

       Anger       0.48      0.46      0.47       105
     Disgust       0.38      0.46      0.41        72
        Fear       0.57      0.75      0.65        73
         Joy       0.60      0.61      0.60       164
     Sadness       0.32      0.53      0.40        64
    Surprise       0.67      0.41      0.51       202

    accuracy                           0.52       680
   macro avg       0.50      0.54      0.51       680
weighted avg       0.55      0.52      0.52       680

Accuracy :  0.5176470588235295
Precison :  0.5389706614646764
Recall :  0.5176470588235295
F1 :  0.5160479398474551


#KNN

In [58]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = knn.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[134  12  17  11   4   1]
 [ 36  26   4  15   5   6]
 [ 31   6  48   6   8   7]
 [ 18   8   7  37   6   1]
 [ 19  12  22   7  57  11]
 [  7   6  14   2  14  56]]
              precision    recall  f1-score   support

           0       0.55      0.75      0.63       179
           1       0.37      0.28      0.32        92
           2       0.43      0.45      0.44       106
           3       0.47      0.48      0.48        77
           4       0.61      0.45      0.51       128
           5       0.68      0.57      0.62        99

    accuracy                           0.53       681
   macro avg       0.52      0.50      0.50       681
weighted avg       0.53      0.53      0.52       681

Accuracy :  0.5256975036710719
Precison :  0.5574612979752361
Recall :  0.5256975036710719
F1 :  0.5328899247068141


In [59]:
test_pred = knn.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [60]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Fear
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Disgust
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Disgust
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Anger
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Disgust
676,677,tomar kukur ta amader onek distrub kore,Anger
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Surprise
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Sadness


In [61]:
df['label'].value_counts()

Joy         217
Surprise    114
Anger       105
Fear         97
Sadness      79
Disgust      68
Name: label, dtype: int64

In [62]:
df.to_csv(folder_path +'becodemixed_knn.tsv',sep = '\t',index = False)

In [63]:
print_metrices(actual_labels['label'],df['label'])

[[ 52  10  14   9   6  14]
 [  5  32   1  10  10  10]
 [  8   3  61   8   9   8]
 [  9  27   4 122  31  24]
 [ 12  13   4  10  35   5]
 [ 15   3  12   8  15  61]]
              precision    recall  f1-score   support

       Anger       0.51      0.50      0.50       105
     Disgust       0.36      0.47      0.41        68
        Fear       0.64      0.63      0.63        97
         Joy       0.73      0.56      0.64       217
     Sadness       0.33      0.44      0.38        79
    Surprise       0.50      0.54      0.52       114

    accuracy                           0.53       680
   macro avg       0.51      0.52      0.51       680
weighted avg       0.56      0.53      0.54       680

Accuracy :  0.5338235294117647
Precison :  0.5263736387324253
Recall :  0.5338235294117647
F1 :  0.5250989430002134


#Adaboost

In [64]:
ada = AdaBoostClassifier(n_estimators=100)
rf.fit(tfidfvectorized_train,train_data['enc_label'])
y_pred = rf.predict(tfidfvectorized_valid)
print_metrices(y_pred,valid_data['enc_label'])

[[105  10  46  12   5   1]
 [ 20  28  14  13  14   3]
 [ 18   1  74   5   4   4]
 [ 23   6   8  29  11   0]
 [  8   6  41   3  66   4]
 [  5   0  25   1  13  55]]
              precision    recall  f1-score   support

           0       0.59      0.59      0.59       179
           1       0.55      0.30      0.39        92
           2       0.36      0.70      0.47       106
           3       0.46      0.38      0.41        77
           4       0.58      0.52      0.55       128
           5       0.82      0.56      0.66        99

    accuracy                           0.52       681
   macro avg       0.56      0.51      0.51       681
weighted avg       0.56      0.52      0.53       681

Accuracy :  0.5242290748898678
Precison :  0.5652634453006476
Recall :  0.5242290748898678
F1 :  0.5218795169993479


In [65]:
test_pred = rf.predict(tfidfvectorized_test).tolist()
test_data['id'] = test_data.index + 1

In [66]:
df = pd.DataFrame(test_data['id'])
df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,text,label
0,1,tumi sagor er eto pare gele keno,Anger
1,2,Your most favourite juti ? ? ? ? Pls vote guys...,Surprise
2,3,etai mone o mitthe bolche 😒 😒 Dev Dev 😏,Joy
3,4,"Ei movie ta dekhe mone holo , ami worst direct...",Fear
4,5,"eita bangladesh eigula hbei,ei desh e teke eig...",Sadness
...,...,...,...
675,676,Nickleodeon animated series ta thik jototai na...,Disgust
676,677,tomar kukur ta amader onek distrub kore,Anger
677,678,Koto expekta equipmpuro kelasness er jonho kha...,Anger
678,679,Ami gorbito ami bangladeshi ai desh e somvob a...,Sadness


In [67]:
df['label'].value_counts()

Surprise    204
Joy         154
Anger       100
Disgust      79
Fear         75
Sadness      68
Name: label, dtype: int64

In [68]:
df.to_csv(folder_path +'becodemixed_adaboost.tsv',sep = '\t',index = False)

In [69]:
print_metrices(actual_labels['label'],df['label'])

[[47  6 14 14 13  6]
 [ 5 35  1 17 13  8]
 [ 3  4 54  2  7  5]
 [ 3 21  3 95 16 16]
 [ 6 10  1 12 35  4]
 [37 12 23 27 22 83]]
              precision    recall  f1-score   support

       Anger       0.47      0.47      0.47       100
     Disgust       0.40      0.44      0.42        79
        Fear       0.56      0.72      0.63        75
         Joy       0.57      0.62      0.59       154
     Sadness       0.33      0.51      0.40        68
    Surprise       0.68      0.41      0.51       204

    accuracy                           0.51       680
   macro avg       0.50      0.53      0.50       680
weighted avg       0.54      0.51      0.51       680

Accuracy :  0.513235294117647
Precison :  0.5335189716514779
Recall :  0.513235294117647
F1 :  0.5123020101623448


##Deep Learning

In [70]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM,GRU
from tensorflow.keras.models import load_model
print("Tensorflow Version:--",tf.version.VERSION)
print("Keras Version:---",keras.__version__)

Tensorflow Version:-- 2.8.2
Keras Version:--- 2.8.0


###Tokenization

In [71]:
tokenizer = Tokenizer(num_words = 10000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n-', 
                      split=' ', char_level=False, oov_token='<oov>', document_count=0)
tokenizer.fit_on_texts(train_data['Text'])  
vocab_size = len(tokenizer.word_index)+1
vocab_size

13932

## what is thattttttt

1.def def pad_sequences(sequences, maxlen=None, dtype='int32', padding='pre', truncating='pre', value=0.0)
Pads sequences to the same length.

This function transforms a list (of length num_samples) of sequences (lists of integers) into a 2D Numpy array of shape (num_samples, num_timesteps). num_timesteps is either the maxlen argument if provided, or the length of the longest sequence in the list.
2.Public API for tf.keras.preprocessing.sequence namespace.



In [72]:
max_len = 300
# Training Sequences
train_sequences = tokenizer.texts_to_sequences(train_data['Text']) #def texts_to_sequences(texts)Transforms each text in texts to a sequence of integers.
print(len(tokenizer.word_index))
train_pad_sequences =  keras.preprocessing.sequence.pad_sequences(train_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Training Sequences :" ,train_pad_sequences.shape)

# Validation Sequences
validation_sequences = tokenizer.texts_to_sequences(valid_data['Text'])
validation_pad_sequences =  keras.preprocessing.sequence.pad_sequences(validation_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Validation Sequences :" ,validation_pad_sequences.shape)

13931
Number of Training Sequences : (5453, 300)
Number of Validation Sequences : (681, 300)


In [73]:
# Test Sequences
test_sequences = tokenizer.texts_to_sequences(test_data['Text'])
test_pad_sequences =  keras.preprocessing.sequence.pad_sequences(test_sequences, value=0.0, padding='post', maxlen= max_len)
print("Number of Test Sequences :" ,test_pad_sequences.shape)

Number of Test Sequences : (680, 300)


In [74]:

# Estimate class weights for unbalanced datasets.

# Parameters

# class_weight : dict, 'balanced' or None
#     If 'balanced', class weights will be given by
# n_samples / (n_classes * np.bincount(y)).
# If a dictionary is given, keys are classes and values
# are corresponding class weights.
# If None is given, the class weights will be uniform.

# classes : ndarray

In [75]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced'
                                               ,classes =np.unique(train_data['enc_label'])
                                               ,y=train_data['enc_label'])

weight = {i : class_weights[i] for i in range(6)}

In [76]:
weight

{0: 0.6677687974528533,
 1: 1.0923477564102564,
 2: 1.1083333333333334,
 3: 1.1460697772173183,
 4: 1.013192121887774,
 5: 1.2117777777777778}

In [77]:
# def def __init__(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch', options=None, **kwargs)
# Callback to save the Keras model or model weights at some frequency.

# ModelCheckpoint callback is used in conjunction with training using model.fit() to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

# A few options this callback provides include:

# Whether to only keep the model that has achieved the "best performance" so far, or whether to save the model at the end of every epoch regardless of performance.
# Definition of 'best'; which quantity to monitor and whether it should be maximized or minimized.
# The frequency it should save at. Currently, the callback supports saving at the end of every epoch, or after a fixed number of training batches.
# Whether only weights are saved, or the whole model is saved.
# Note: If you get WARNING:tensorflow:Can save best model only with <name> available, skipping see the description of the monitor argument for details on how to get this right.


# def def __init__(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, **kwargs)
# Turns positive integers (indexes) into dense vectors of fixed size.

# e.g. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

# This layer can only be used as the first layer in a model.

# Example:

# >>> model = tf.keras.Sequential()
# >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
# >>> # The model will take as input an integer matrix of size (batch,
# >>> # input_length), and the largest integer (i.e. word index) in the input
# >>> # should be no larger than 999 (vocabulary size).
# >>> # Now model.output_shape is (None, 10, 64), where `None` is the batch
# >>> # dimension.

##cnn

In [128]:
num_classes = 6

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_cnn.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
# def def __init__(filters, kernel_size, strides=1, padding='valid', data_format='channels_last', dilation_rate=1, groups=1, activation=None, use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros', kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, bias_constraint=None, **kwargs)
# 1D convolution layer (e.g. temporal convolution).
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.MaxPooling1D(pool_size=2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 conv1d_3 (Conv1D)           (None, 296, 128)          64128     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 148, 128)         0         
 1D)                                                             
                                                                 
 flatten_6 (Flatten)         (None, 18944)             0         
                                                                 
 dense_6 (Dense)             (None, 6)                 113670    
                                                                 
Total params: 1,177,798
Trainable params: 1,177,798
Non-trainable params: 0
____________________________________________

In [129]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
136/137 [============================>.] - ETA: 0s - loss: 1.5594 - accuracy: 0.3667
Epoch 1: val_accuracy improved from -inf to 0.51054, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_cnn.h5
137/137 [==============================] - 12s 83ms/step - loss: 1.5590 - accuracy: 0.3668 - val_loss: 1.3089 - val_accuracy: 0.5105
Epoch 2/30
136/137 [============================>.] - ETA: 0s - loss: 0.6930 - accuracy: 0.7567
Epoch 2: val_accuracy improved from 0.51054 to 0.52796, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_cnn.h5
137/137 [==============================] - 9s 67ms/step - loss: 0.6936 - accuracy: 0.7561 - val_loss: 1.3585 - val_accuracy: 0.5280
Epoch 3/30
136/137 [============================>.] - ETA: 0s - loss: 0.1657 - accuracy: 0.9545
Epoch 3: val_accuracy improved from 0.52796 to 0.55545, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_cnn.h5
137/137 [==============================] - 9s 67ms/step -

In [130]:
# Load the saved model
model = load_model(folder_path+'becodemixed_cnn.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[122  28  17  15  12   6]
 [ 14  26   2   9  10   1]
 [ 19   3  61   3  22   4]
 [ 17  24   5  40  12   6]
 [  4   9   8   8  56  11]
 [  3   2  13   2  16  71]]
              precision    recall  f1-score   support

           0       0.68      0.61      0.64       200
           1       0.28      0.42      0.34        62
           2       0.58      0.54      0.56       112
           3       0.52      0.38      0.44       104
           4       0.44      0.58      0.50        96
           5       0.72      0.66      0.69       107

    accuracy                           0.55       681
   macro avg       0.54      0.53      0.53       681
weighted avg       0.57      0.55      0.56       681

Accuracy :  0.5521292217327459
Precison :  0.5513603936645098
Recall :  0.5521292217327459
F1 :  0.5461116140104787


In [131]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Anger
1,2,Fear
2,3,Disgust
3,4,Disgust
4,5,Anger
...,...,...
675,676,Disgust
676,677,Anger
677,678,Disgust
678,679,Disgust


In [132]:
df['label'].value_counts()

Joy         169
Anger       115
Surprise    114
Disgust     106
Fear        105
Sadness      71
Name: label, dtype: int64

In [133]:
print_metrices(actual_labels['label'],df['label'])

[[ 61   5  12  12   9  16]
 [  9  52   0  17  22   6]
 [  5   3  67   6   5  19]
 [  6  17   3 103  27  13]
 [  9   8   3  12  39   0]
 [ 11   3  11  17   4  68]]
              precision    recall  f1-score   support

       Anger       0.60      0.53      0.56       115
     Disgust       0.59      0.49      0.54       106
        Fear       0.70      0.64      0.67       105
         Joy       0.62      0.61      0.61       169
     Sadness       0.37      0.55      0.44        71
    Surprise       0.56      0.60      0.58       114

    accuracy                           0.57       680
   macro avg       0.57      0.57      0.57       680
weighted avg       0.59      0.57      0.58       680

Accuracy :  0.5735294117647058
Precison :  0.5746753150696629
Recall :  0.5735294117647058
F1 :  0.5700373646216209


In [84]:

df.to_csv(folder_path +'becodemixed_cnnfinal.tsv',sep = '\t',index = False)

#CNN_AVI_IMP

In [85]:
# embedding_dim = 30

# model = Sequential()
# model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
# # model.add(layers.Embedding(vocab_size, embedding_dim, 
# #                            weights=[embedding_matrix], 
# #                            input_length=maxlen, 
# #                            trainable=False))
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.MaxPooling1D())
# model.add(layers.Conv1D(32, 5, activation='relu'))
# model.add(layers.Conv1D(128, 5, activation='relu'))
# model.add(layers.MaxPooling1D())
# model.add(layers.Flatten())
# model.add(layers.Dense(32, activation='relu'))
# # model.add(layers.Dropout(0.4))
# model.add(layers.Dense(6, activation='softmax'))

## lstm

In [134]:
num_classes = 6
from keras.layers.embeddings import Embedding
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_LSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
# lstm
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),

tf.keras.layers.LSTM(units = 128,activation='tanh',return_sequences=True,dropout = 0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()



Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 lstm_5 (LSTM)               (None, 300, 128)          117248    
                                                                 
 flatten_7 (Flatten)         (None, 38400)             0         
                                                                 
 dense_7 (Dense)             (None, 6)                 230406    
                                                                 
Total params: 1,347,654
Trainable params: 1,347,654
Non-trainable params: 0
_________________________________________________________________


In [135]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
137/137 [==============================] - ETA: 0s - loss: 1.5172 - accuracy: 0.3794
Epoch 1: val_accuracy improved from -inf to 0.55820, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_LSTM.h5
137/137 [==============================] - 49s 337ms/step - loss: 1.5172 - accuracy: 0.3794 - val_loss: 1.2037 - val_accuracy: 0.5582
Epoch 2/30
137/137 [==============================] - ETA: 0s - loss: 0.6210 - accuracy: 0.7909
Epoch 2: val_accuracy did not improve from 0.55820
137/137 [==============================] - 46s 333ms/step - loss: 0.6210 - accuracy: 0.7909 - val_loss: 1.3341 - val_accuracy: 0.5545
Epoch 3/30
137/137 [==============================] - ETA: 0s - loss: 0.1954 - accuracy: 0.9413
Epoch 3: val_accuracy improved from 0.55820 to 0.57745, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_LSTM.h5
137/137 [==============================] - 47s 346ms/step - loss: 0.1954 - accuracy: 0.9413 - val_loss: 1.7508 - val_accuracy: 0.577

In [136]:
# Load the saved model
model = load_model(folder_path+'becodemixed_LSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[127  29  16  12  11   5]
 [ 16  30   3  12  11   7]
 [ 11   2  53   1  16   8]
 [ 23  23  17  43  22   6]
 [  1   8   6   8  54  15]
 [  1   0  11   1  14  58]]
              precision    recall  f1-score   support

           0       0.71      0.64      0.67       200
           1       0.33      0.38      0.35        79
           2       0.50      0.58      0.54        91
           3       0.56      0.32      0.41       134
           4       0.42      0.59      0.49        92
           5       0.59      0.68      0.63        85

    accuracy                           0.54       681
   macro avg       0.52      0.53      0.51       681
weighted avg       0.55      0.54      0.53       681

Accuracy :  0.5359765051395007
Precison :  0.5546700496034995
Recall :  0.5359765051395007
F1 :  0.5373165768074513


In [137]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Anger
1,2,Fear
2,3,Disgust
3,4,Disgust
4,5,Anger
...,...,...
675,676,Disgust
676,677,Anger
677,678,Disgust
678,679,Disgust


In [138]:
df['label'].value_counts()

Joy         178
Disgust     132
Anger       105
Sadness      93
Fear         86
Surprise     86
Name: label, dtype: int64

In [139]:
print_metrices(actual_labels['label'],df['label'])

[[ 58   4  13   6   6  18]
 [ 14  56   6  17  24  15]
 [  4   2  62   4   4  10]
 [  4  18   2 115  24  15]
 [ 11   8   4  20  45   5]
 [ 10   0   9   5   3  59]]
              precision    recall  f1-score   support

       Anger       0.57      0.55      0.56       105
     Disgust       0.64      0.42      0.51       132
        Fear       0.65      0.72      0.68        86
         Joy       0.69      0.65      0.67       178
     Sadness       0.42      0.48      0.45        93
    Surprise       0.48      0.69      0.57        86

    accuracy                           0.58       680
   macro avg       0.58      0.59      0.57       680
weighted avg       0.59      0.58      0.58       680

Accuracy :  0.5808823529411765
Precison :  0.5959034812876575
Recall :  0.5808823529411765
F1 :  0.5817131619142567


## bilstm

In [140]:
num_classes = 6

accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_BiLSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
#tf.keras.layers.Conv1D(128, 5, activation='relu'),
#tf.keras.layers.MaxPooling1D(5),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 300, 100)          1000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 300, 256)         234496    
 nal)                                                            
                                                                 
 flatten_8 (Flatten)         (None, 76800)             0         
                                                                 
 dense_8 (Dense)             (None, 6)                 460806    
                                                                 
Total params: 1,695,302
Trainable params: 1,695,302
Non-trainable params: 0
_________________________________________________________________


In [141]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
137/137 [==============================] - ETA: 0s - loss: 1.5116 - accuracy: 0.3916
Epoch 1: val_accuracy improved from -inf to 0.50962, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_BiLSTM.h5
137/137 [==============================] - 68s 469ms/step - loss: 1.5116 - accuracy: 0.3916 - val_loss: 1.2662 - val_accuracy: 0.5096
Epoch 2/30
137/137 [==============================] - ETA: 0s - loss: 0.5734 - accuracy: 0.7989
Epoch 2: val_accuracy improved from 0.50962 to 0.55820, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_BiLSTM.h5
137/137 [==============================] - 64s 465ms/step - loss: 0.5734 - accuracy: 0.7989 - val_loss: 1.4138 - val_accuracy: 0.5582
Epoch 3/30
137/137 [==============================] - ETA: 0s - loss: 0.1638 - accuracy: 0.9546
Epoch 3: val_accuracy improved from 0.55820 to 0.57195, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_BiLSTM.h5
137/137 [==============================] - 65

In [142]:
# Load the saved model
model = load_model(folder_path+'becodemixed_BiLSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[137  28  24  15  21   8]
 [ 14  37   1  11  10   0]
 [ 11   1  49   2   8   7]
 [ 15  20   6  37   7   2]
 [  2   6  12  10  68  15]
 [  0   0  14   2  14  67]]
              precision    recall  f1-score   support

           0       0.77      0.59      0.67       233
           1       0.40      0.51      0.45        73
           2       0.46      0.63      0.53        78
           3       0.48      0.43      0.45        87
           4       0.53      0.60      0.56       113
           5       0.68      0.69      0.68        97

    accuracy                           0.58       681
   macro avg       0.55      0.57      0.56       681
weighted avg       0.60      0.58      0.59       681

Accuracy :  0.580029368575624
Precison :  0.5824139882863268
Recall :  0.580029368575624
F1 :  0.5747735068630914


In [143]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Anger
1,2,Fear
2,3,Disgust
3,4,Sadness
4,5,Anger
...,...,...
675,676,Disgust
676,677,Anger
677,678,Anger
678,679,Sadness


In [144]:
df['label'].value_counts()

Joy         187
Anger       123
Fear        111
Sadness      91
Disgust      89
Surprise     79
Name: label, dtype: int64

In [145]:
print_metrices(actual_labels['label'],df['label'])

[[ 62   5  14   8   9  25]
 [  7  41   0  19  13   9]
 [  8   5  68   8  10  12]
 [  6  22   3 110  27  19]
 [ 10  15   3  15  46   2]
 [  8   0   8   7   1  55]]
              precision    recall  f1-score   support

       Anger       0.61      0.50      0.55       123
     Disgust       0.47      0.46      0.46        89
        Fear       0.71      0.61      0.66       111
         Joy       0.66      0.59      0.62       187
     Sadness       0.43      0.51      0.47        91
    Surprise       0.45      0.70      0.55        79

    accuracy                           0.56       680
   macro avg       0.56      0.56      0.55       680
weighted avg       0.58      0.56      0.56       680

Accuracy :  0.5617647058823529
Precison :  0.5691398916470921
Recall :  0.5617647058823529
F1 :  0.5585375846992783


#cnn-lstm

In [149]:
num_classes = 6
from keras.layers.embeddings import Embedding
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_CNN-LSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
# lstm
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.MaxPooling1D(pool_size=2),
tf.keras.layers.LSTM(units = 128,activation='tanh',return_sequences=True,dropout = 0.2),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()



Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 conv1d_5 (Conv1D)           (None, 296, 128)          64128     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 148, 128)         0         
 1D)                                                             
                                                                 
 lstm_8 (LSTM)               (None, 148, 128)          131584    
                                                                 
 flatten_10 (Flatten)        (None, 18944)             0         
                                                                 
 dense_10 (Dense)            (None, 6)                 113670    
                                                     

In [150]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
137/137 [==============================] - ETA: 0s - loss: 1.5834 - accuracy: 0.3226
Epoch 1: val_accuracy improved from -inf to 0.51146, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_CNN-LSTM.h5
137/137 [==============================] - 34s 232ms/step - loss: 1.5834 - accuracy: 0.3226 - val_loss: 1.2886 - val_accuracy: 0.5115
Epoch 2/30
137/137 [==============================] - ETA: 0s - loss: 0.8380 - accuracy: 0.6942
Epoch 2: val_accuracy improved from 0.51146 to 0.53071, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_CNN-LSTM.h5
137/137 [==============================] - 31s 230ms/step - loss: 0.8380 - accuracy: 0.6942 - val_loss: 1.3405 - val_accuracy: 0.5307
Epoch 3/30
137/137 [==============================] - ETA: 0s - loss: 0.2971 - accuracy: 0.9035
Epoch 3: val_accuracy did not improve from 0.53071
137/137 [==============================] - 31s 225ms/step - loss: 0.2971 - accuracy: 0.9035 - val_loss: 1.7540 - val_accurac

In [151]:
# Load the saved model
model = load_model(folder_path+'becodemixed_CNN-LSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[112  27  18  13   4   6]
 [ 22  39   7  11  13   9]
 [ 14   2  51   5  18   3]
 [ 22  13   8  35  16   3]
 [  5   9  13  12  58  13]
 [  4   2   9   1  19  65]]
              precision    recall  f1-score   support

           0       0.63      0.62      0.62       180
           1       0.42      0.39      0.40       101
           2       0.48      0.55      0.51        93
           3       0.45      0.36      0.40        97
           4       0.45      0.53      0.49       110
           5       0.66      0.65      0.65       100

    accuracy                           0.53       681
   macro avg       0.52      0.52      0.51       681
weighted avg       0.53      0.53      0.53       681

Accuracy :  0.5286343612334802
Precison :  0.5354713315075449
Recall :  0.5286343612334802
F1 :  0.530452218320281


In [152]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Anger
1,2,Fear
2,3,Disgust
3,4,Disgust
4,5,Anger
...,...,...
675,676,Disgust
676,677,Anger
677,678,Anger
678,679,Disgust


In [153]:
df['label'].value_counts()

Joy         163
Anger       123
Sadness     109
Disgust     104
Fear         96
Surprise     85
Name: label, dtype: int64

In [154]:
print_metrices(actual_labels['label'],df['label'])

[[ 55   8  13  14  14  19]
 [ 10  46   2  15  20  11]
 [  7   1  66   5   5  12]
 [  4  17   3 105  13  21]
 [ 14  13   3  21  50   8]
 [ 11   3   9   7   4  51]]
              precision    recall  f1-score   support

       Anger       0.54      0.45      0.49       123
     Disgust       0.52      0.44      0.48       104
        Fear       0.69      0.69      0.69        96
         Joy       0.63      0.64      0.64       163
     Sadness       0.47      0.46      0.47       109
    Surprise       0.42      0.60      0.49        85

    accuracy                           0.55       680
   macro avg       0.55      0.55      0.54       680
weighted avg       0.55      0.55      0.55       680

Accuracy :  0.5485294117647059
Precison :  0.5580679689362746
Recall :  0.5485294117647059
F1 :  0.5491998174051309


#CNN-BiLSTM

In [160]:
num_classes = 6
from keras.layers.embeddings import Embedding
accuracy_threshold = 0.99

class myCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>accuracy_threshold):
        print("\nReached %2.2f%% accuracy so we will stop trianing" % (accuracy_threshold*100))
        self.model.stop_training = True

acc_callback = myCallback()
# Saved the Best Model
filepath = folder_path+"becodemixed_CNN-BiLSTM.h5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=2, save_best_only=True, 
                                             save_weights_only=False, mode='max')
# callback list
callback_list = [acc_callback, checkpoint] 
# lstm
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000, 100, input_length = max_len),
tf.keras.layers.Conv1D(128, 5, activation='relu'),
tf.keras.layers.MaxPooling1D(pool_size=2),
tf.keras.layers.Bidirectional(LSTM(units = 128,return_sequences=True,dropout = 0.2)),
tf.keras.layers.Flatten(),
tf.keras.layers.Dense(6 , activation='softmax')])

model.summary()



Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 300, 100)          1000000   
                                                                 
 conv1d_6 (Conv1D)           (None, 296, 128)          64128     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 148, 128)         0         
 1D)                                                             
                                                                 
 bidirectional_4 (Bidirectio  (None, 148, 256)         263168    
 nal)                                                            
                                                                 
 flatten_11 (Flatten)        (None, 37888)             0         
                                                                 
 dense_11 (Dense)            (None, 6)               

In [161]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.003),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
   
model.fit(train_pad_sequences,train_data['enc_label'], epochs=30, batch_size = 32, validation_split = 0.2,verbose =1,callbacks = callback_list,class_weight = weight)

Epoch 1/30
137/137 [==============================] - ETA: 0s - loss: 1.6480 - accuracy: 0.2898
Epoch 1: val_accuracy improved from -inf to 0.39047, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_CNN-BiLSTM.h5
137/137 [==============================] - 46s 313ms/step - loss: 1.6480 - accuracy: 0.2898 - val_loss: 1.4715 - val_accuracy: 0.3905
Epoch 2/30
137/137 [==============================] - ETA: 0s - loss: 1.0940 - accuracy: 0.5825
Epoch 2: val_accuracy improved from 0.39047 to 0.49771, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_CNN-BiLSTM.h5
137/137 [==============================] - 43s 313ms/step - loss: 1.0940 - accuracy: 0.5825 - val_loss: 1.3485 - val_accuracy: 0.4977
Epoch 3/30
137/137 [==============================] - ETA: 0s - loss: 0.4321 - accuracy: 0.8528
Epoch 3: val_accuracy improved from 0.49771 to 0.52887, saving model to /content/drive/MyDrive/model_imp_6kdata/becodemixed_CNN-BiLSTM.h5
137/137 [========================

In [162]:
# Load the saved model
model = load_model(folder_path+'becodemixed_CNN-BiLSTM.h5')
# prediction
y_pred = np.argmax(model.predict(validation_pad_sequences), axis=-1)#numpy.argmax(a, axis=None, out=None, *, keepdims=<no value>)Returns the indices of the maximum values along an axis.
print_metrices(valid_data['enc_label'],y_pred)


[[97 26 12  7  4  3]
 [18 26  3 12  8  3]
 [27  1 66  5 18  8]
 [19 16  7 40 12  2]
 [18 23 12 13 76 27]
 [ 0  0  6  0 10 56]]
              precision    recall  f1-score   support

           0       0.54      0.65      0.59       149
           1       0.28      0.37      0.32        70
           2       0.62      0.53      0.57       125
           3       0.52      0.42      0.46        96
           4       0.59      0.45      0.51       169
           5       0.57      0.78      0.65        72

    accuracy                           0.53       681
   macro avg       0.52      0.53      0.52       681
weighted avg       0.54      0.53      0.53       681

Accuracy :  0.5301027900146843
Precison :  0.5481866276425236
Recall :  0.5301027900146843
F1 :  0.5314709297742026


In [163]:
test_pred = np.argmax(model.predict(test_pad_sequences), axis=-1).tolist()
# test_pred
test_data['id'] = test_data.index + 1
df = pd.DataFrame(test_data['id'])
# df['text'] = test_data['Text']
df['label'] = test_pred
df['label'] = df['label'].apply(actual_label)
df

,id,label
0,1,Anger
1,2,Fear
2,3,Disgust
3,4,Anger
4,5,Anger
...,...,...
675,676,Disgust
676,677,Anger
677,678,Anger
678,679,Sadness


In [164]:
df['label'].value_counts()

Anger       188
Joy         119
Surprise    112
Sadness      94
Disgust      86
Fear         81
Name: label, dtype: int64

In [165]:
print_metrices(actual_labels['label'],df['label'])

[[69 20 18 24 29 28]
 [ 6 36  1 19 16  8]
 [ 6  0 62  5  0  8]
 [ 1 11  0 83 14 10]
 [ 7 18  4 22 41  2]
 [12  3 11 14  6 66]]
              precision    recall  f1-score   support

       Anger       0.68      0.37      0.48       188
     Disgust       0.41      0.42      0.41        86
        Fear       0.65      0.77      0.70        81
         Joy       0.50      0.70      0.58       119
     Sadness       0.39      0.44      0.41        94
    Surprise       0.54      0.59      0.56       112

    accuracy                           0.53       680
   macro avg       0.53      0.55      0.52       680
weighted avg       0.55      0.53      0.52       680

Accuracy :  0.525
Precison :  0.5617554867833303
Recall :  0.525
F1 :  0.5310396865361598


#DATA STAT

In [4]:
%%time
becodemixed_data =  pd.read_excel('/content/drive/MyDrive/model data stat/be_data_no_un_combined.xlsx') 


CPU times: user 624 ms, sys: 27.4 ms, total: 651 ms
Wall time: 1.07 s


In [5]:
becodemixed_data['enc_label'] = becodemixed_data['Label'].replace({'Neutral' :6 ,'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})

In [6]:
print("Size of the dataset ==> ", becodemixed_data.shape)

Size of the dataset ==>  (10221, 3)


In [7]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [8]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(becodemixed_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

train_data= pd.concat([becodemixed_data, encoded_labels ], axis=1)

In [10]:
becodemixed_data = becodemixed_data.rename(columns={'Label_Neutral' :'Neutral' ,'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
becodemixed_data.head(5)

,Label,Text,enc_label
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3
1,Disgust,Romantic comedy te je kikore eto vulgar jinish...,3
2,Sadness,Shokal thake burger r grill khaite mon chai bu...,1
3,Joy,jakhon dekhbi banglar chhele-meye banglay par...,0
4,Disgust,Mymensingh er manush eto kharap kn : ( na obos...,3


In [12]:
becodemixed_data['Label'].value_counts()

Joy         2575
Anger       1677
Surprise    1571
Sadness     1558
Disgust     1430
Fear        1410
Name: Label, dtype: int64

In [13]:
becodemixed_data['enc_label'] = becodemixed_data['Label'].replace({'Joy': 0,'Disgust':  3,'Anger' :4,
                                                       'Sadness': 1,'Surprise':2,'Fear': 5})


In [14]:
becodemixed_data.head(1)

,Label,Text,enc_label
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3


In [15]:
becodemixed_data['enc_label'].value_counts()

0    2575
4    1677
2    1571
1    1558
3    1430
5    1410
Name: enc_label, dtype: int64

In [16]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [17]:
encoder=OneHotEncoder(sparse=False)

encoded_labels = pd.DataFrame (encoder.fit_transform(becodemixed_data[['Label']]))

encoded_labels .columns = encoder.get_feature_names(['Label'])

becodemixed_data= pd.concat([becodemixed_data, encoded_labels ], axis=1)

In [18]:
becodemixed_data = becodemixed_data.rename(columns={'Label_Joy': 'Joy',
                                        'Label_Disgust':  'Disgust','Label_Anger' :'Anger',
                                        'Label_Sadness': 'Sadness',
                                        'Label_Surprise':'Surprise','Label_Fear': 'Fear'
                                      
                                      })
becodemixed_data.head(1)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3,0.0,1.0,0.0,0.0,0.0,0.0


In [19]:
'''
Text Cleaning
'''
def text_cleaning(row):
  # to remove HTML tags
  text = BeautifulSoup(row, 'html.parser').get_text()
  d = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) #This line is for removing url
  post = d.replace('\n', '')
  post = post.replace('—', ' ')
  # to remove accented characters
  new_text = unicodedata.normalize('NFKD', post).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  # to remove special characters and numbers
  # define the pattern to keep
  #pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
  #text = re.sub(pat, '', new_text)
  #to remove punctuation
  text = ''.join([c for c in text if c not in string.punctuation])
  # to remove special characters
  pattern = r'^\s*|\s\s*'
  text = re.sub(pattern, ' ', text).strip()
  # convert into lower case
  text = text.lower() 
  # Stopword Removing
  tokenizer = ToktokTokenizer()
  # convert sentence into token of words
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  # check in lowercase 
  #t = [token for token in tokens if token.lower() not in stopword_list]
  #text = ' '.join(t) 
  # Stemming
  #t = [stemming.stem(token) for token in tokens ]
  #text = ' '.join(t) 

  return text



In [21]:
def get_becodemixed_stop_words():
    #swdf1 = pd.read_csv("/content/drive/MyDrive/automated_tamil_text_classification-master/data/TamilNLP_TamilStopWords.txt",  header=None) 
    swdf2 = pd.read_csv("/content/drive/MyDrive/model data stat/custom_becodemixed_stopwords.txt",  header=None) 
    #sw1 = swdf1[0].tolist()
    sw2 = swdf2[0].tolist()
    #tamil_stop_words = list(set(sw1 + sw2))
    becodemixed_stop_words = list(sw2)
    return becodemixed_stop_words

In [22]:
# #removing URLs and Punctuations
# becodemixed_data['cleaned'] = becodemixed_data['Texts'].apply(remove_url)
# #Removing punctuations


becodemixed_stop_words = get_becodemixed_stop_words()
becodemixed_data['Texts'] = becodemixed_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
becodemixed_data['cleaned'] = becodemixed_data['Texts'].apply(text_cleaning)


# valid_data['Texts'] = valid_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
# train_data['Texts'] = train_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))
# test_data['Texts'] = test_data['Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (becodemixed_stop_words)]))

# train_data['cleaned'] = train_data['Texts'].apply(text_cleaning)
# valid_data['cleaned'] = valid_data['Texts'].apply(text_cleaning)
# test_data['cleaned'] = test_data['Texts'].apply(text_cleaning)


## Data samples after cleaning
print("Original Data:===\n",becodemixed_data.Text[0],"\nCleaned Data:===\n",becodemixed_data.cleaned[0],)

Original Data:===
  Dakhachish ai to tui svf are chala..to jeet dar collection to fake blban e to ... sir manush boka noi 
Cleaned Data:===
 dakhachish tui svf are chalato jeet dar collection fake blban sir manush boka noi


In [23]:
df=becodemixed_data 
df['word_count'] = df.cleaned.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 150
df3 = df[more_than_150]
df3

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count


In [24]:
def char_length(row):
    return len(row.split())

In [25]:
lables = ['Joy','Sadness','Surprise','Disgust','Anger','Fear']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    

In [26]:
becodemixed_data.head(2)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count
0,Disgust,Dakhachish ai to tui svf are chala..to jeet d...,3,0.0,1.0,0.0,0.0,0.0,0.0,Dakhachish tui svf are chala..to jeet dar coll...,dakhachish tui svf are chalato jeet dar collec...,14
1,Disgust,Romantic comedy te je kikore eto vulgar jinish...,3,0.0,1.0,0.0,0.0,0.0,0.0,Romantic comedy kikore vulgar jinish potro dho...,romantic comedy kikore vulgar jinish potro dho...,10


In [27]:
dataset_statistics(becodemixed_data)


Class Name:--> Joy 
Number of Words:31333
Number of Unique Words:6773
Maximum post length: 79
Average Words in texts: 12.168155339805825

Class Name:--> Sadness 
Number of Words:18988
Number of Unique Words:4801
Maximum post length: 53
Average Words in texts: 12.187419768934532

Class Name:--> Surprise 
Number of Words:10460
Number of Unique Words:3256
Maximum post length: 26
Average Words in texts: 6.658179503500955

Class Name:--> Disgust 
Number of Words:17762
Number of Unique Words:4703
Maximum post length: 43
Average Words in texts: 12.42097902097902

Class Name:--> Anger 
Number of Words:14573
Number of Unique Words:3970
Maximum post length: 77
Average Words in texts: 8.689922480620154

Class Name:--> Fear 
Number of Words:11049
Number of Unique Words:2841
Maximum post length: 23
Average Words in texts: 7.836170212765958


# Data Statistics

In [117]:
df=train_data 
df['word_count'] = df.cleaned.apply(lambda x: len(str(x).split()))
more_than_150 =  df['word_count'] > 150
df3 = df[more_than_150]
df3

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count


In [118]:
def char_length(row):
    return len(row.split())

In [119]:
lables = ['Joy','Sadness','Surprise','Disgust','Anger','Fear']

# Count number of unique words in each set
def dataset_statistics(dataset):
  for l in lables:
    word_list = [word.strip() for t in list(dataset[dataset[l]==1.0].cleaned) for word in t.strip().split()]
    print(f"\nClass Name:--> {l} ") 
    print("Number of Words:{}".format(len(word_list))) 
    print("Number of Unique Words:{}".format(len(np.unique(word_list)))) 
    print("Maximum post length: {}".format(max(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    print("Average Words in texts: {}".format(np.mean(dataset[dataset[l]==1.0].cleaned.apply(char_length))))
    

In [120]:
train_data.head(2)

,Label,Text,enc_label,Anger,Disgust,Fear,Joy,Sadness,Surprise,Texts,cleaned,word_count
0,Joy,Jurassic Park er ekta sequel Jurassic World je...,0,0.0,0.0,0.0,1.0,0.0,0.0,Jurassic Park sequel Jurassic World gengineere...,jurassic park sequel jurassic world gengineere...,19
1,Surprise,E . T movie ta eto captivating ar endearingly ...,2,0.0,0.0,0.0,0.0,0.0,1.0,E . T movie captivating ar endearingly optimis...,e t movie captivating ar endearingly optimisti...,13


In [121]:
dataset_statistics(train_data)


Class Name:--> Joy 
Number of Words:16654
Number of Unique Words:5869
Maximum post length: 79
Average Words in texts: 12.236590742101397

Class Name:--> Sadness 
Number of Words:10207
Number of Unique Words:4204
Maximum post length: 53
Average Words in texts: 12.268028846153847

Class Name:--> Surprise 
Number of Words:5542
Number of Unique Words:2730
Maximum post length: 26
Average Words in texts: 6.758536585365854

Class Name:--> Disgust 
Number of Words:9749
Number of Unique Words:4110
Maximum post length: 43
Average Words in texts: 12.293820933165195

Class Name:--> Anger 
Number of Words:7771
Number of Unique Words:3388
Maximum post length: 40
Average Words in texts: 8.663322185061315

Class Name:--> Fear 
Number of Words:5927
Number of Unique Words:2443
Maximum post length: 23
Average Words in texts: 7.902666666666667
